In [19]:
import tensorflow as tf
import numpy as np

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 使用TensorFlow训练DNN

### 构建阶段

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [4]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z

In [5]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")

#### 用dense()替换neuron_layer()函数

In [6]:
from tensorflow.layers import dense

"""
tf.layers.dense(
    inputs,
    units,
    activation=None,
    use_bias=True,
    kernel_initializer=None,
    bias_initializer=tf.zeros_initializer(),
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    trainable=True,
    name=None,
    reuse=None
)
"""

with tf.name_scope("dnn"):
    hidden1 = dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = dense(hidden2, n_outputs, name="outputs")

In [7]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [8]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 执行阶段

In [12]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../dataset/mnist/")

Extracting ../dataset/mnist/train-images-idx3-ubyte.gz
Extracting ../dataset/mnist/train-labels-idx1-ubyte.gz
Extracting ../dataset/mnist/t10k-images-idx3-ubyte.gz
Extracting ../dataset/mnist/t10k-labels-idx1-ubyte.gz


In [13]:
mnist.train.num_examples

55000

In [14]:
n_epochs = 40
batch_size = 50

In [15]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                            y: mnist.test.labels})
        print(epoch, "Train acc:", acc_train, "Test acc:", acc_test)
    
    save_path = saver.save(sess, "./model/my_model_final.ckpt")

0 Train acc: 0.76 Test acc: 0.9045
1 Train acc: 0.84 Test acc: 0.9224
2 Train acc: 0.9 Test acc: 0.9294
3 Train acc: 0.98 Test acc: 0.9362
4 Train acc: 0.96 Test acc: 0.9427
5 Train acc: 1.0 Test acc: 0.9454
6 Train acc: 0.94 Test acc: 0.9503
7 Train acc: 0.96 Test acc: 0.9522
8 Train acc: 0.96 Test acc: 0.9558
9 Train acc: 0.92 Test acc: 0.9574
10 Train acc: 0.96 Test acc: 0.9593
11 Train acc: 0.96 Test acc: 0.9617
12 Train acc: 0.94 Test acc: 0.9644
13 Train acc: 0.96 Test acc: 0.9661
14 Train acc: 0.96 Test acc: 0.9676
15 Train acc: 0.96 Test acc: 0.9676
16 Train acc: 0.96 Test acc: 0.9674
17 Train acc: 0.98 Test acc: 0.9699
18 Train acc: 0.98 Test acc: 0.9696
19 Train acc: 1.0 Test acc: 0.9719
20 Train acc: 1.0 Test acc: 0.9709
21 Train acc: 1.0 Test acc: 0.9728
22 Train acc: 1.0 Test acc: 0.9741
23 Train acc: 0.98 Test acc: 0.9721
24 Train acc: 0.98 Test acc: 0.9736
25 Train acc: 1.0 Test acc: 0.9738
26 Train acc: 0.98 Test acc: 0.9752
27 Train acc: 0.98 Test acc: 0.9737
28 Train 

# 使用神经网络

In [16]:
with tf.Session() as sess:
    saver.restore(sess, "./model/my_model_final.ckpt")
    X_new_scaled = mnist.test.images[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
print(y_pred)
print(mnist.test.labels[:20])

INFO:tensorflow:Restoring parameters from ./model/my_model_final.ckpt
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


# 完整代码

In [23]:
reset_graph()

In [25]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

mnist = input_data.read_data_sets("../dataset/mnist/")

n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                            y: mnist.test.labels})
        print(epoch, "Train acc:", acc_train, "Test acc:", acc_test)
    
    #save_path = saver.save(sess, "./model/my_model_final.ckpt")

Extracting ../dataset/mnist/train-images-idx3-ubyte.gz
Extracting ../dataset/mnist/train-labels-idx1-ubyte.gz
Extracting ../dataset/mnist/t10k-images-idx3-ubyte.gz
Extracting ../dataset/mnist/t10k-labels-idx1-ubyte.gz
0 Train acc: 0.88 Test acc: 0.9064
1 Train acc: 0.94 Test acc: 0.9218
2 Train acc: 0.92 Test acc: 0.9304
3 Train acc: 0.96 Test acc: 0.9378
4 Train acc: 0.92 Test acc: 0.9438
5 Train acc: 0.92 Test acc: 0.9443
6 Train acc: 0.96 Test acc: 0.9486
7 Train acc: 0.94 Test acc: 0.9532
8 Train acc: 0.96 Test acc: 0.9555
9 Train acc: 0.98 Test acc: 0.9583
10 Train acc: 0.98 Test acc: 0.9598
11 Train acc: 0.92 Test acc: 0.9618
12 Train acc: 1.0 Test acc: 0.9632
13 Train acc: 0.96 Test acc: 0.9629
14 Train acc: 1.0 Test acc: 0.9663
15 Train acc: 1.0 Test acc: 0.9667
16 Train acc: 1.0 Test acc: 0.9676
17 Train acc: 0.96 Test acc: 0.9674
18 Train acc: 1.0 Test acc: 0.9698
19 Train acc: 0.98 Test acc: 0.9687
20 Train acc: 1.0 Test acc: 0.9706
21 Train acc: 1.0 Test acc: 0.971
22 Train